In [53]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json

In [28]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name)
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

In [3]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

base.json file이 모든 정보를 가지고 있으므로 이화일만 다운로드
그 후 내용 화일로 분리

In [4]:
s3 = boto3.client('s3')
content_object = s3.get_object(Bucket = 'dsmapdata', Key = User_Name + '/base.json')
file_content = content_object['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

In [5]:
for key_ in json_content.keys():
  if 'user' not in key_ :
    df = pd.json_normalize(json_content, record_path =[key_])
    file_to_save = os.path.join(src_folder ,'{}.json'.format(key_))
    print(file_to_save)
    df.to_json(open(file_to_save, 'w', encoding='utf-8'),orient="records",force_ascii=False)

./Downloaded\dsadmin\area_def.json
./Downloaded\dsadmin\course_info.json


작업.

작업이 완료되면 UPload folder로 이동

In [11]:

import shutil

files = glob.glob(os.path.join(src_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]

for f_ in target_files:
  shutil.copy(f_, out_folder)

In [67]:
files = glob.glob(os.path.join(out_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

['./ToUpload\\dsadmin\\area_def.json', './ToUpload\\dsadmin\\course_info.json']

각 화일을 base.json으로 통합, 그후 가가 화일과 통합화일을 Upload

In [77]:
Total_json = {}


for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key='baseinfo/{}.json'.format(f_))
  
s3.put_object(
  Body=json.dumps(Total_json).encode('utf-8'),
  Bucket='dsmapdata',
  Key=User_Name + '/base.json'
  )

{'ResponseMetadata': {'RequestId': 'V2JS6662P8SFZ7Y6',
  'HostId': '3twkefgjqNCc0zT83U1rml9625TJcd4cfOk4YpbcOlfn3r6Osi1StAH6G5+alUXpmx+68knIdo020+/oYMm42g==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3twkefgjqNCc0zT83U1rml9625TJcd4cfOk4YpbcOlfn3r6Osi1StAH6G5+alUXpmx+68knIdo020+/oYMm42g==',
   'x-amz-request-id': 'V2JS6662P8SFZ7Y6',
   'date': 'Wed, 09 Aug 2023 03:10:24 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f7690d507adfbda7284973e37f5afac5"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f7690d507adfbda7284973e37f5afac5"',
 'ServerSideEncryption': 'AES256'}

In [78]:
Total_json

{'area_def': [{'TypeId': 0,
   'name': 'Undefined',
   'color': '#8d99ae',
   'display': True,
   'DSZindex': 100},
  {'TypeId': 1,
   'name': '전면',
   'color': '#00a0e9',
   'display': True,
   'DSZindex': 1},
  {'TypeId': 2,
   'name': '코스',
   'color': '#B71C1C',
   'display': True,
   'DSZindex': 2},
  {'TypeId': 3,
   'name': '홀영역',
   'color': '#1A237E',
   'display': True,
   'DSZindex': 3},
  {'TypeId': 4,
   'name': '그린',
   'color': '#336600',
   'display': True,
   'DSZindex': 4},
  {'TypeId': 5,
   'name': '티박스',
   'color': '#0D47A1',
   'display': True,
   'DSZindex': 5},
  {'TypeId': 6,
   'name': '훼어웨이',
   'color': '#FFFF33',
   'display': True,
   'DSZindex': 6},
  {'TypeId': 7,
   'name': '기타관심지역',
   'color': '#d90429',
   'display': True,
   'DSZindex': 200}],
 'course_info': [{'id': 'MGC001',
   'dscourseids': '',
   'name': '올림픽CC',
   'address': '경기 고양시 덕양구 혜음로 301',
   'numHole': 9,
   'course_names': ['올림픽'],
   'map_info': {'center': [126.894265298, 37.723108